In [1]:
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

+ pip install konlpy
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 44.1 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
+ curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh
+ bash -x
+ mecab_dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic
+ set -e
++ uname
+ os=Linux
+ [[ ! Linux == \L\i\n\u\x ]]
+ hash sudo
+ sudo=sudo
+ python=python3
+ hash pyenv
+ at_user_site=
++ check_python_site_location_is_writable
++ python3 -
+ [[ 1 == \0 ]]
+ hash automake
+ echo 'Installing automake (A dependency for mecab-ko)'
Installing automake (A dependency for mecab-ko)
+ install_automake
+ '[' Linux == Linux ']'
++ grep -Ei 'debian|buntu|mint' /etc/lsb-release /etc/os-release
+ '[' '/etc/lsb-release:DISTRIB_ID=Ubuntu
/etc/l

In [2]:
cur_dir = '/content/drive/MyDrive/KLUE_TC/data/open/'

In [3]:
import pandas as pd

In [4]:
dacon_train_data = pd.read_csv(cur_dir + 'train_data.csv')
dacon_topic_data = pd.read_csv(cur_dir + 'topic_dict.csv')
dacon_test_data = pd.read_csv(cur_dir + 'test_data.csv')

In [5]:
import konlpy

mecab = konlpy.tag.Mecab()

In [6]:
topic = sorted(list(dacon_train_data['topic_idx'].unique()))

In [7]:
topic_df = []

for num in topic:
    topic_df.append(list(dacon_train_data[dacon_train_data['topic_idx'] == num]['title']))

In [8]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 4.1 MB/s 
     |████████████████████████████████| 636 kB 34.6 MB/s 
     |████████████████████████████████| 3.3 MB 49.5 MB/s 
     |████████████████████████████████| 895 kB 58.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [9]:
from tqdm import tqdm
from transformers import AutoTokenizer

In [10]:
klue_roberta_tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large')

In [11]:
topic_df_tokenized = []
for corpus in tqdm(topic_df):
    temp = []
    for text in corpus:
        temp.append(klue_roberta_tokenizer.convert_ids_to_tokens(klue_roberta_tokenizer(text)['input_ids'], skip_special_tokens=True))
    topic_df_tokenized.append(temp)

100%|██████████| 7/7 [00:22<00:00,  3.21s/it]


In [12]:
topic_df_tokenized

[[['지',
   '##카',
   '##바이',
   '##러스',
   '##도',
   '규명',
   '##한',
   '초저',
   '##온',
   '##전자',
   '##현',
   '##미',
   '##경',
   '…',
   '신약',
   '##연구',
   '##에',
   '유용'],
  ['증강현실', '①', '알파', '##고', '이어', '포켓몬', '고', '…', '거센', 'IT', '광풍'],
  ['AI', '월드컵', '생생', '##한', '현장', '중계', '##도'],
  ['이세돌',
   '알파',
   '##고',
   '집중력',
   '사람',
   '##이',
   '이기',
   '##긴',
   '어렵',
   '##다',
   '일문일답',
   '##종합',
   '##2',
   '##보'],
  ['올해',
   '휴대폰',
   '국내',
   '생산량',
   '2',
   '##천',
   '##50',
   '##0',
   '##만',
   '##대',
   '…',
   '10',
   '##년',
   '##전',
   '##의',
   '18',
   '.',
   '4',
   '%'],
  ['LG',
   '갤',
   '##노',
   '##트',
   '##8',
   '겨냥',
   '##한',
   'V',
   '##30',
   '티저',
   '광고',
   '2',
   '##편',
   '##으로',
   '선전',
   '##포',
   '##고'],
  ['KT',
   '##스카',
   '##이',
   '##라이',
   '##프',
   '알뜰',
   '##폰',
   '판매',
   '…',
   '전용',
   '요금제',
   '6',
   '##종',
   '출시'],
  ['안',
   '쓰',
   '##는',
   'TV',
   '주파수',
   '농어촌',
   '무선',
   '##인',
   '##터',
   '##

In [13]:
import re, math

n = 7

tf_list = []
w_list = []

In [14]:
def tf_count(words, i):
    for word in words:
      if(word != ''):
        if word in tf_list[i]:
            tf_list[i][word] += 1
        else:
            tf_list[i][word] = 1

In [15]:
for i, corpus in enumerate(topic_df_tokenized):
  for words in corpus:
    tf_list.append(dict())
    tf_count(words, i)  
    
  print("Text", i+1 , "words:", words)
  print("Text", i+1 , "counts:", tf_list[i])

for i in range(0, 7):
  w_list.append(dict())
  for word in tf_list[i]:
    df = 0
    for j in range(0, 7):
      if word in tf_list[j]:
        df += 1
    w_list[i][word] = tf_list[i][word] * math.log(n/df)
  
  #정렬 규칙 : 이름 순 정렬, w값 (dictionary의 value값) 내림차순 정렬

  w_list[i] = sorted(w_list[i].items())
  w_list[i] = sorted(w_list[i], key=(lambda x: x[1]), reverse=True)
  
  print('Text' , i+1 , 'Result')
  for j in range(0, 100):
    print(w_list[i][j])
  print(w_list[i])
  print('===================================================')

Output hidden; open in https://colab.research.google.com to view.

In [16]:
vocabs = []

for idx, element in enumerate(w_list):
    l = int(len(element)/2)
    vocab = []
    for data in element[:l]:
        vocab.append(data[0])
    vocabs.append(vocab)
    print(l)

3617
3806
5269
4634
5015
3819
4230


In [17]:
def writeDict(file_name, dictionary):
    with open(file_name, 'w', encoding='utf-8') as file:
        for data in dictionary:
            file.write(data + '\n')

In [ ]:
for idx, element in enumerate(vocabs):
    for idx_1, element_1 in enumerate(vocabs):
        if idx != idx_1:
            element, element_1 = set(element), set(element_1)
            if len(element & element_1) > 0:
                gyo_set = element & element_1
                # if idx_1 == 2:
                #     element = element - gyo_set
                # elif idx == 2:
                #     element_1 = element_1 - gyo_set
                # else:
                #     element = element - gyo_set
                #     element_1 = element_1 - gyo_set
                element = element - gyo_set
                element_1 = element_1 - gyo_set
                vocabs[idx] = list(element)
                vocabs[idx_1] = list(element_1)
                print(f"{idx}과 {idx_1} : {element & element_1}")
    print("\n\n==========================================\n\n")

0과 1 : set()
0과 2 : set()
0과 3 : set()
0과 4 : set()
0과 5 : set()
0과 6 : set()




1과 2 : set()
1과 3 : set()
1과 4 : set()
1과 5 : set()
1과 6 : set()




2과 3 : set()
2과 4 : set()
2과 5 : set()
2과 6 : set()




3과 4 : set()
3과 5 : set()
3과 6 : set()




4과 5 : set()
4과 6 : set()




5과 6 : set()










In [ ]:
print(vocabs)

[['##IST', '과학자', '와이어', '사라진다', 'FM', '##기능', '산학', '폴더', '배양', '##슬링', '실생활', '노화', '클릭', '행성', '바이러스', '##쥐', '위주', '화학', '후후', '액정', '수증기', '나노', '메신저', '업데이트', '##해졌', '듀얼', 'ICT', '통신망', '알려준다', '##화질', '##개발', '급성장', '최적화', '##테리아', '액세서리', '기능', '접착', '알약', '알림', '액티브', '##브스', '궁금증', '세기', '문어', '연락처', '노트북', '혈압', '##큐', '항암제', '곰팡이', '척척', '수퍼', '급속', 'X', '해라', '##떨', '증후군', '싸이월드', '생쥐', '알츠하이머', '감지', '구글', '탑재', '쥐', '미생물', '홈피', '##넷', '##폼', '코엑스', '박테리아', 'Y', '원자', '어벤져스', '발산', '##y', '접목', '타사', '글라스', 'OLED', '와이파이', '용접', '카카오톡', '제원', '동급', '야심', '클로', '접속', '키즈', '배터리', '송출', '워치', '바둑', '이식', '도매', '게이머', '발화', '도메인', '증식', '기기', '버전', '구별', '윈도', '트랜', '넘나들', '아차', '효소', '직영', '음양', '##인드', '##탭', '에너지원', '128', '##OS', '손쉽', '戰', '촉매', '미모', '보상', '선명', '자폐', '벌거', '개시', '고분', '해저', '업그레이드', '##클리', '알렉', '웨어', '높인', '초음파', '아이즈', '검색', '치료법', 'PC', 'RPG', '림프', '아이언맨', '후면', '세균', '##캠', '브라우저', '가성', '##TE', '수납', '화면', '간암', '본다', '고도', '세포', 'IF', '완성도',

In [18]:
for idx, element in enumerate(vocabs):
    print(len(element))
    writeDict('/content/drive/MyDrive/KLUE_TC/data/vocab/vocab50/' + str(idx)+'.txt', element)

3617
3806
5269
4634
5015
3819
4230


In [ ]:
for idx, element in enumerate(vocabs):
    print(element)
    print("\n\n=======================================\n\n")

['##IST', '과학자', '와이어', '사라진다', 'FM', '##기능', '산학', '폴더', '배양', '##슬링', '실생활', '노화', '클릭', '행성', '바이러스', '##쥐', '위주', '화학', '후후', '액정', '수증기', '나노', '메신저', '업데이트', '##해졌', '듀얼', 'ICT', '통신망', '알려준다', '##화질', '##개발', '급성장', '최적화', '##테리아', '액세서리', '기능', '접착', '알약', '알림', '액티브', '##브스', '궁금증', '세기', '문어', '연락처', '노트북', '혈압', '##큐', '항암제', '곰팡이', '척척', '수퍼', '급속', 'X', '해라', '##떨', '증후군', '싸이월드', '생쥐', '알츠하이머', '감지', '구글', '탑재', '쥐', '미생물', '홈피', '##넷', '##폼', '코엑스', '박테리아', 'Y', '원자', '어벤져스', '발산', '##y', '접목', '타사', '글라스', 'OLED', '와이파이', '용접', '카카오톡', '제원', '동급', '야심', '클로', '접속', '키즈', '배터리', '송출', '워치', '바둑', '이식', '도매', '게이머', '발화', '도메인', '증식', '기기', '버전', '구별', '윈도', '트랜', '넘나들', '아차', '효소', '직영', '음양', '##인드', '##탭', '에너지원', '128', '##OS', '손쉽', '戰', '촉매', '미모', '보상', '선명', '자폐', '벌거', '개시', '고분', '해저', '업그레이드', '##클리', '알렉', '웨어', '높인', '초음파', '아이즈', '검색', '치료법', 'PC', 'RPG', '림프', '아이언맨', '후면', '세균', '##캠', '브라우저', '가성', '##TE', '수납', '화면', '간암', '본다', '고도', '세포', 'IF', '완성도', 